In [1]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
import math
from tensorflow.python.framework import ops

In [2]:
# load attributes 
data_dir = os.getcwd() + '\\Data\\'
train_data = pd.read_csv(data_dir + 'train.csv')
train_labels = pd.read_csv(data_dir + 'train_labels.csv')
train_label_bits = np.genfromtxt(data_dir + 'train_label_bits.csv', delimiter=',', dtype=np.float32) 

# loading revised images
image_dir = data_dir + 'images\\res_rev_05\\' 

# total count of samples
sample_count = 1584

# load images
images = []
for image_label in range(sample_count):
    image = mpimg.imread(image_dir + str(image_label + 1) + '.jpg')
    images.append(image)
    
images = np.array(images)

train_images = images[train_data['id'] - 1]
train_images = np.reshape(train_images, (len(train_data), images[0].shape[0], images[0].shape[1], -1))
train_images.shape

(990, 85, 54, 1)

In [3]:
TOTAL_COUNT = len(images)
TRAIN_COUNT = len(train_data) # training image count in input csv file
IMAGE_WIDTH  = images[0].shape[0] # input image width
IMAGE_HEIGHT = images[0].shape[1] # input image height
OUTPUT_TYPE_COUNT = len(list(set(train_data['species'])))
print('Image Count: ', TOTAL_COUNT)
print('Image Size: ', IMAGE_WIDTH , 'X' , IMAGE_HEIGHT)
print('Train Count: ', TRAIN_COUNT)
print('Specy Count: ', OUTPUT_TYPE_COUNT)

Image Count:  1584
Image Size:  85 X 54
Train Count:  990
Specy Count:  99


In [4]:
TEST_COUNT = 190 # splitting data to test the model
train_X = train_images[:-TEST_COUNT]
test_X = train_images[-TEST_COUNT:]
train_Y = train_label_bits[:-TEST_COUNT]
test_Y = train_label_bits[-TEST_COUNT:]

In [5]:
# reading other features
temp = train_data.copy().sort_values(by = ['species']).copy()
margins = temp.values[:,2:66].copy()
train_margins = margins[:-TEST_COUNT]
test_margins = margins[-TEST_COUNT:]

shapes = temp.values[:,66:130].copy()
train_shapes = shapes[:-TEST_COUNT]
test_shapes = shapes[-TEST_COUNT:]

textures =temp.values[:,130:199].copy()
train_textures = textures[:-TEST_COUNT]
test_textures = textures[-TEST_COUNT:]

In [6]:
# the batch size equals to BATCH_SIZE it will
# starts from 0 to until the last index of
# batch equals to TOTAL_COUNT
BATCH_COUNTER = 0

def next_batch(all_images, margins, shapes, textures, all_labels):
    
    # accessing BATCH_COUNTER inside function
    global BATCH_COUNTER
    
    # check if exceeds to the last index then reset batch_counter
    if BATCH_COUNTER == len(all_labels) / BATCH_SIZE:
        BATCH_COUNTER = 0

    # setting first and last index of data
    index_from = BATCH_COUNTER * BATCH_SIZE
    index_to   = (BATCH_COUNTER + 1) * BATCH_SIZE
    
    # loading a batch of training images and labels
    image_batch = all_images[index_from:index_to]
    shape_batch = shapes[index_from:index_to]
    margin_batch = margins[index_from:index_to]
    texture_batch = textures[index_from:index_to]
    label_batch = all_labels[index_from:index_to]
    
    # incrementing batch_counter for the next iteration
    BATCH_COUNTER += 1 
    
    return image_batch, margin_batch, shape_batch, texture_batch, label_batch;

In [7]:
ITERATION_COUNT = 1000
ACCURACY_CHECK = 100
BATCH_SIZE = 100
PKEEP = 0.75

C0 = 1   # input channel count
C1 = 4   # convolutional network channel 1 count 
C2 = 8   # convolutional network channel 2 count
C3 = 12   # convolutional network channel 3 count
C4 = 64  # fulley connected layer size
C5 = OUTPUT_TYPE_COUNT  # output count 

# Probability of keeping a node during dropout 
pkeep = tf.placeholder(tf.float32)

def compatible_convolutional_noise_shape(Y):
    noiseshape = tf.shape(Y)
    noiseshape = noiseshape * tf.constant([1,0,0,1]) + tf.constant([0,1,1,0])
    return noiseshape

# weights
W1 = tf.Variable(tf.truncated_normal([2, 2, C0, C1], stddev = 0.1)) 
W2 = tf.Variable(tf.truncated_normal([5, 5, C1, C2], stddev = 0.1))
W3 = tf.Variable(tf.truncated_normal([4, 4, C2, C3], stddev = 0.1))
W4 = tf.Variable(tf.truncated_normal([11 * 7 * C3 , C4], stddev = 0.1))
W5 = tf.Variable(tf.truncated_normal([C4 * 4, C5], stddev = 0.1))

# biases
B1 = tf.Variable(tf.ones([C1]) / 10)
B2 = tf.Variable(tf.ones([C2]) / 10)
B3 = tf.Variable(tf.ones([C3]) / 10)
B4 = tf.Variable(tf.ones([C4]) / 10)
B5 = tf.Variable(tf.ones([C5]) / 10)

# model
stride1 = 1  # output is 85x54
X   = tf.placeholder(tf.float32, shape=[None, IMAGE_WIDTH, IMAGE_HEIGHT, C0])
Y1  = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride1, stride1, 1], padding='SAME') + B1)
Y1m = tf.nn.max_pool(Y1,ksize = [1, 2, 2, 1], strides = [1, 1, 1, 1], padding='SAME')
Y1d = tf.nn.dropout(Y1, pkeep, compatible_convolutional_noise_shape(Y1))
stride2 = 4  # output is 43x27
Y2 = tf.nn.relu(tf.nn.conv2d(Y1m, W2, strides=[1, stride2, stride2, 1], padding='SAME') + B2)
Y2m = tf.nn.max_pool(Y2,ksize = [1, 2, 2, 1], strides = [1, 1, 1, 1], padding='SAME')
#Y2 = tf.nn.dropout(Y2r, pkeep, compatible_convolutional_noise_shape(Y2m))
stride3 = 2  # output is 22x14
Y3 = tf.nn.relu(tf.nn.conv2d(Y2m, W3, strides=[1, stride3, stride3, 1], padding='SAME') + B3)
Y3m = tf.nn.max_pool(Y3,ksize = [1, 2, 2, 1], strides = [1, 1, 1, 1], padding='SAME')
Y3d = tf.nn.dropout(Y3m, pkeep, compatible_convolutional_noise_shape(Y3m))

# reshape the output from the third convolution for the fully connected layer
YY = tf.reshape(Y3m, shape=[-1, 11 * 7 * C3])
Y4 = tf.nn.relu(tf.matmul(YY, W4) + B4, name='4')

# concating other feature to flattened layer
MAR = tf.placeholder(tf.float32, shape=[None, 64])
SHP = tf.placeholder(tf.float32, shape=[None, 64])
TXT = tf.placeholder(tf.float32, shape=[None, 64])
STK = tf.stack([Y4, MAR, SHP, TXT], -1, name='stack') 

# softmax layer
Y4_ = tf.reshape(STK, (-1, 64 * 4), name='reshape')
Ylogits = tf.matmul(Y4_, W5) + B5
Y = tf.nn.softmax(Ylogits, name='5')

# paceholder for learing_rate variable
lr = tf.placeholder(tf.float32)
    
# placeholder for correct answers
Y_ = tf.placeholder(tf.float32, [None, C5])
    
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = Ylogits, labels = Y_)
cross_entropy = tf.reduce_mean(cross_entropy) * 100
    
# accuracy of the trained model, between 0 (worst) and 1 (best)
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
# training step, the learning rate is a placeholder
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

# initialisation & start computation
init = tf.global_variables_initializer()
sess = tf.Session()
tf.reset_default_graph()
tf.set_random_seed(0)
sess.run(init)
    
for i in range(ITERATION_COUNT):
    
    # learning rate decay
    max_learning_rate = 0.003
    min_learning_rate = 0.0001
    decay_speed = 2000.0 
    learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
        
    # load batch of images and correct answers
    batch_X , mar, shp, txt, batch_Y = next_batch(train_X, train_margins, train_shapes, train_shapes, train_Y)
        
    # train
    sess.run(train_step, feed_dict = {X: batch_X, Y_: batch_Y, MAR: mar, SHP: shp, TXT: txt, lr: learning_rate, pkeep: PKEEP})
    
    if (i + 1) % ACCURACY_CHECK == 0:

        # train validation
        a , c = sess.run([accuracy, cross_entropy], feed_dict = {X: batch_X, Y_: batch_Y, MAR: mar, SHP: shp, TXT: txt, pkeep: 1.0})
        print('Train:', i + 1, ":" , a, c, learning_rate)
        
        # test validation
        a , c = sess.run([accuracy, cross_entropy], feed_dict = {X: test_X, Y_: test_Y, MAR: test_margins, SHP: test_shapes, TXT: test_textures, pkeep: 1.0})
        print('Test:', i + 1, ":" , a, c, learning_rate)
    
a, c, y1, w1, y, y1d, y1m, yy, y4, ylogits, y_, y2, y2m, y3, y3m, y3d, w2, w3, w4, y4_, stk, w5, b5,b4 = sess.run([accuracy, cross_entropy, Y1, W1, Y, Y1d, Y1m, YY, Y4, Ylogits, Y_, Y2, Y2m, Y3, Y3m, Y3d, W2, W3, W4, Y4_, STK, W5, B5,B4], feed_dict =  {X: test_X, Y_: test_Y, MAR: test_margins, SHP: test_shapes, TXT: test_textures, pkeep: 1.0})


Train: 100 : 0.91 44.0667 0.0028599449585957402
Test: 100 : 0.405263 261.886 0.0028599449585957402
Train: 200 : 0.99 5.98062 0.0027253408546186733
Test: 200 : 0.410526 375.765 0.0027253408546186733
Train: 300 : 1.0 1.08807 0.002597301470257133
Test: 300 : 0.447368 468.232 0.002597301470257133
Train: 400 : 1.0 4.94578 0.00247550664035748
Test: 400 : 0.389474 430.95 0.00247550664035748
Train: 500 : 1.0 0.114193 0.00235965181440487
Test: 500 : 0.447368 497.51 0.00235965181440487
Train: 600 : 1.0 0.0867965 0.0022494472949883388
Test: 600 : 0.457895 522.431 0.0022494472949883388
Train: 700 : 1.0 0.0424672 0.002144617513406374
Test: 700 : 0.468421 539.893 0.002144617513406374
Train: 800 : 1.0 0.0450315 0.002044900340601626
Test: 800 : 0.478947 553.898 0.002044900340601626
Train: 900 : 1.0 0.0260005 0.0019500464317017275
Test: 900 : 0.478947 565.506 0.0019500464317017275
Train: 1000 : 1.0 0.029477 0.001859818602527234
Test: 1000 : 0.478947 575.264 0.001859818602527234


In [8]:
print(y4.shape, w4.shape, y4_.shape, w5.shape, b5.shape, b4.shape, stk.shape)


(190, 64) (924, 64) (190, 256) (256, 99) (99,) (64,) (190, 64, 4)


In [9]:
sess.close()